# Crypto Currency API Project
## Perodically Extract
* Create a system which periodically extracts, updates and store the **list of crypto exchanges** from https://www.coingecko.com/en/exchanges
* Apply country and type filter
    * **Country:** Australia, Croatia, Cyprus, Czech Republic, Estonia, Hong Kong, Ireland, Japan, Malta, Netherlands, Norway, Poland, Singapore, South Korea, Switzerland, United Kingdom, United States
    * **Type:** Centralized
* Find all the **currencies that are available on the filtered exchanges** (Use Coinmarketcap.com and Coingecko.com for the same)
* Create a system that will check and update the number of **filtered exchanges each of the following coins** is listed on:
    * Bitcoin
    * Ethereum
    * Litecoin
    * NEO
    * Stellar
    * Ethereum Classic

### Before we begin

From this [source](https://news.bitcoin.com/the-number-of-cryptocurrency-exchanges-has-exploded/) we know that there are about 500 crpyto currency exchanges in the world.

* On [Coinhills](https://www.coinhills.com/market/exchange/) there are about 224 exchanges and 2,000 (5,521) currencies.
* On [CoinMarketCap](https://coinmarketcap.com/) there are about 263 exchanges and 2,440 currencies.
* On [CoinGecko](https://www.coingecko.com/en/exchanges) there are about 369 exchanges and 5,410 coins.

Further research on CoinGecko on how the list of exchanges have grown on CoinGecko, since this is the website we would be focusing on for this task ([source](https://bit.ly/CoinGecko-2019-Q2-Report)).

<img src="files/image.png">

We would be mostly using [CoinGecko's API Wrapper](https://github.com/man-c/pycoingecko) for this task.

### Let's begin...

In [1]:
# Importing packages
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd, requests, json, selenium, time
from pycoingecko import CoinGeckoAPI
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
# Let's explore get_exchanges_list API wrapper
cg = CoinGeckoAPI()
exchanges_list = cg.get_exchanges_list()
exchanges_list[0:4]

[{'id': 'kraken',
  'name': 'Kraken',
  'year_established': 2011,
  'country': 'United States',
  'description': None,
  'url': 'https://www.kraken.com/en-us',
  'image': 'https://assets.coingecko.com/markets/images/29/small/kraken.jpg?1519460351',
  'has_trading_incentive': False,
  'trust_score': 10,
  'trust_score_rank': 1,
  'trade_volume_24h_btc': 5763.565946758953,
  'trade_volume_24h_btc_normalized': 5740.276531712762},
 {'id': 'bitfinex',
  'name': 'Bitfinex',
  'year_established': 2014,
  'country': 'British Virgin Islands',
  'description': None,
  'url': 'https://www.bitfinex.com',
  'image': 'https://assets.coingecko.com/markets/images/4/small/bitfinex.jpg?1519355988',
  'has_trading_incentive': None,
  'trust_score': 10,
  'trust_score_rank': 2,
  'trade_volume_24h_btc': 5068.1704269330485,
  'trade_volume_24h_btc_normalized': 5068.1704269330485},
 {'id': 'kucoin',
  'name': 'KuCoin',
  'year_established': 2014,
  'country': 'Seychelles',
  'description': '',
  'url': 'htt

For the sub task 1 we get the name of the country, however we don't get the type of exchange i.e. whether it's centralized or decentralized. For 'Vinex' we see that it's mentioned in the description. Exploring more on [request URL](https://api.coingecko.com/api/v3/exchanges) we see there are 4 mentions for centralized and 2 mentions for decentralized exchanges, which is not enough.

### Extract list of exchanges (along with id) and country from the above `exchanges_list`

In [3]:
# Get list of exchanges
def get_list_of_exchanges():
    cg = CoinGeckoAPI()
    exchanges_list = cg.get_exchanges_list()
    get_exchanges_id = []
    get_exchanges_list = []
    get_exchanges_country = []

    # Converting the 'exchanges_list' to a dictionary.
    exchanges_list = {item['name']:item for item in exchanges_list}

    for k, v in exchanges_list.items():
        get_exchanges_id.append(v['id'])
        get_exchanges_list.append(v['name'])
        get_exchanges_country.append(v['country'])
        
    get_exchanges = pd.DataFrame([get_exchanges_id, get_exchanges_list, get_exchanges_country], 
                             index = ['ID', 'Exchange', 'Country']).T
    
    return get_exchanges

In [4]:
exchanges = get_list_of_exchanges()

print(exchanges.shape)
print(exchanges.head())

(100, 3)
         ID  Exchange                 Country
0    kraken    Kraken           United States
1  bitfinex  Bitfinex  British Virgin Islands
2    kucoin    KuCoin              Seychelles
3      gate   Gate.io               Hong Kong
4  poloniex  Poloniex           United States


Using Coin Gecko API Wrapper we get **~300** exchanges (which keeps changing) whereas the website mentioned 369 exchanges. Checking for the differences.

In [5]:
# Using another wrapper from Coin Gecko to get List of all supported markets id and name
exchanges_id_name = cg.get_exchanges_id_name_list()
exchanges_id_name = {item['name']:item for item in exchanges_id_name}

get_names = []
for k, v in exchanges_id_name.items():
    get_names.append(v['name'])

# Checking the differences in both the lists
diff = set(get_names) - set(exchanges['Exchange'].tolist())

print(len(get_names), '\n\n', diff, '\n\n', len(diff))

386 

 {'Paro Exchange', 'OpenLedger DEX', 'Coinpark', 'CITEX', 'Binance JEX (Futures)', 'Bitfex', 'EOSex', 'OTCBTC', 'FatBTC', 'Coinmetro', 'SatowalletEx', 'Whitebit', 'CoinAll', 'DDEX', 'Everbloom', 'Bitshares Assets', 'Bitinfi', 'CoinBene', 'Paymium', 'Beaxy', 'MXC', 'Coinsuper', 'BiHODL ', 'Upbit Indonesia ', 'Dobitrade', 'Fisco', 'Exrates', 'Saturn Network', 'Vbitex', 'SouthXchange', 'Bitc3', 'Letsdocoinz', 'Dex-Trade', 'AlterDice', 'Bgogo', 'COSS', 'LocalTrade', 'OEX', 'ExMarkets', '9coin', 'TRXMarket', 'Bisq', 'Bitz Futures', 'BTSE', 'BitRabbit', 'BIG markets', 'Bione', 'F1CX', 'Huobi Korea', 'Cryptology', 'CoinAsset', 'BITEXBOOK', 'Velic', 'Coinfinit', 'Coingi', 'CODEX', 'Waves Platform', 'Birake', 'CryTrEx', 'Bilaxy', 'CCXCanada', 'Cryptonit', 'Simex', 'Aphelion', 'Binance Futures', 'Worldcore', 'Coinfalcon', 'Orderbook.io', 'BTCC', 'Newdex', 'Braziliex', 'Coinplace', 'IncoreX', 'Atomars', 'Paribu', 'Dakuce', 'Cobinhood', 'Raisex', 'Ironex', 'Altmarkets', 'MercuriEx', 'Blockon

Most of the aboves there are those where '24h Volume (Normalized)', '24h Volume', 'Visits (SimilarWeb)', 'Coins', and 'Pairs' are 0s. As of now, for the first subtasks, we have list of exchanges (along with id) and country in `get_exchanges` dataframe.

### Create a function that would get the status updates of the exchanges

In [6]:
# def get_exchange_updates():
#     exchanges_status_updates = {}
#     cg = CoinGeckoAPI()
#     for ids in exchanges['ID'].tolist():
#         try: exchanges_status_updates[ids] = cg.get_exchanges_status_updates_by_id(id = ids)
#         except: pass
        
#     return exchanges_status_updates

# exchanges_status_updates = get_exchange_updates()
# print(len(exchanges_status_updates.keys()), '\n', {k: exchanges_status_updates[k] for k in list(exchanges_status_updates)[:2]})

### Filter by list of countries

In [7]:
# Since we're only interested in the selected list of countries
list_selected_countries = ['Australia', 'Croatia', 'Cyprus', 'Czech Republic', 'Estonia', 'Hong Kong', 
                     'Ireland', 'Japan', 'Malta', 'Netherlands', 'Norway', 'Poland', 'Singapore', 
                     'South Korea', 'Switzerland', 'United Kingdom', 'United States']

filtered_exchanges = exchanges[exchanges['Country'].isin(list_selected_countries)]
filtered_exchanges['Country'].value_counts()

United States     8
United Kingdom    7
South Korea       7
Japan             6
Singapore         6
Estonia           6
Malta             5
Hong Kong         4
Australia         3
Switzerland       2
Cyprus            1
Poland            1
Name: Country, dtype: int64

No Ireland exchanges currently on Coin Gecko.

### Scrape the 'type' using 'BeautifulSoup' from the spot and futures exchange urls

In [8]:
def get_spot_records(url = 'https://www.coingecko.com/en/exchanges?view=social'):
    '''
    Function to extract list of spot exchanges from the url along with the type
    '''
    html_spot = requests.get(url).text
    soup_spot = BeautifulSoup(html_spot, 'html.parser')
    s_records = []
    for contents in soup_spot.find('table', class_ = 'sort table mb-0 text-sm text-lg-normal table-scrollable').find_all('a'):
        name_of_exchange = contents.text.strip()
        name_of_type = contents.find_next('small').contents[0].strip()
        s_records.append((name_of_exchange, name_of_type))       
    return s_records

def get_futures_records(url = 'https://www.coingecko.com/en/exchanges/futures?view=social'):   
    '''
    Function to extract list of futures exchanges from the url along with type
    '''
    html_futures = requests.get(url).text
    soup_futures = BeautifulSoup(html_futures, 'html.parser')
    f_records = []
    for contents in soup_futures.find('table', 
                                      class_ = 'sort table mb-0 text-sm text-lg-normal table-scrollable').find_all('a'):
        name_of_exchange = contents.text.strip()
        name_of_type = contents.find_next('small').contents[0].strip()
        f_records.append((name_of_exchange, name_of_type))   
    return f_records

if __name__ == '__main__':
    s_records, f_records = get_spot_records(), get_futures_records()
    s_records = pd.DataFrame(s_records, columns = ['Exchange', 'Type'])
    f_records = pd.DataFrame(f_records, columns = ['Exchange', 'Type'])
    records = pd.concat([s_records, f_records]).reset_index(drop = True)

### Merge `filtered_exchanges` and `records` to create a filtered dataframe (by country and type) 

In [9]:
filtered_exchanges = filtered_exchanges.merge(records, on = ['Exchange'], how = 'left')

filtered_exchanges = filtered_exchanges[filtered_exchanges['Type'].isin(['Centralized'])]

display(filtered_exchanges.head(), filtered_exchanges.describe())

,ID,Exchange,Country,Type
0,kraken,Kraken,United States,Centralized
1,gate,Gate.io,Hong Kong,Centralized
2,poloniex,Poloniex,United States,Centralized
3,bittrex,Bittrex,United States,Centralized
4,binance,Binance,Malta,Centralized


,ID,Exchange,Country,Type
count,55,55,55,55
unique,55,55,12,1
top,bitsdaq,Darb Finance,United States,Centralized
freq,1,1,8,55


### Create function to find all the currencies that are available on the filtered exchanges

In [10]:
def get_ids_coins():
    ids_and_coins = {}
    for ids in filtered_exchanges['ID'].tolist():
        print('Get list of coins for ID: {}'.format(ids))
        try:
            url = 'https://www.coingecko.com/en/exchanges/' + ids
            html_spot = requests.get(url).text
            soup_spot = BeautifulSoup(html_spot, 'html.parser')
            coins = []
            for c in soup_spot.find('table', class_ = 'table table-scrollable').find_all('span', class_ = 'jp-no-break'):
                if c.text.strip() in coins: pass
                else: coins.append(c.text.strip())
                ids_and_coins[ids] = coins
        except: ids_and_coins[ids] = 'NA'
    ids_and_coins = pd.DataFrame(list(ids_and_coins.items()))
    ids_and_coins.columns = ['ID', 'Currency']
    return ids_and_coins

ids_and_coins = get_ids_coins()
filtered_exchanges = filtered_exchanges.merge(ids_and_coins, on = 'ID', how = 'left')
filtered_exchanges.head()

Get list of coins for ID: kraken
Get list of coins for ID: gate
Get list of coins for ID: poloniex
Get list of coins for ID: bittrex
Get list of coins for ID: binance
Get list of coins for ID: bithumb
Get list of coins for ID: gdax
Get list of coins for ID: upbit
Get list of coins for ID: bitflyer
Get list of coins for ID: bitbank
Get list of coins for ID: zaif
Get list of coins for ID: huobi_us
Get list of coins for ID: bitstamp
Get list of coins for ID: hitbtc
Get list of coins for ID: coinone
Get list of coins for ID: coinsbit
Get list of coins for ID: exmo
Get list of coins for ID: bitbay
Get list of coins for ID: omgfin
Get list of coins for ID: gemini
Get list of coins for ID: tokenize
Get list of coins for ID: luno
Get list of coins for ID: btcmarkets
Get list of coins for ID: bw
Get list of coins for ID: dcoin
Get list of coins for ID: coincheck
Get list of coins for ID: bitmax
Get list of coins for ID: quoine
Get list of coins for ID: gopax
Get list of coins for ID: korbit
Get

,ID,Exchange,Country,Type,Currency
0,kraken,Kraken,United States,Centralized,"[Bitcoin, Ethereum, XRP, Tether, Litecoin, Bit..."
1,gate,Gate.io,Hong Kong,Centralized,"[Bitcoin, Ethereum, EOS, SERO, XRP, BlockCDN, ..."
2,poloniex,Poloniex,United States,Centralized,"[Bitcoin, XRP, Ethereum, Litecoin, Bitcoin Cas..."
3,bittrex,Bittrex,United States,Centralized,"[Bitcoin, XRP, Ethereum, Ravencoin, Bitcoin Ca..."
4,binance,Binance,Malta,Centralized,"[Bitcoin, Ethereum, Dock, XRP, Binance Coin, E..."


### Create a function for number of filtered exchanges for the selected list of coins

In [11]:
def get_markets_coins():
    list_selected_coins = {'bitcoin': 'Bitcoin', 'ethereum': 'Ethereum', 
                       'litecoin': 'Litecoin', 'neo': 'NEO', 'stellar': 'Stellar', 
                       'ethereum-classic': 'Ethereum Classic'}
    get_tickers = []
    for ids in list_selected_coins:
        tickers_by_id = requests.get('https://api.coingecko.com/api/v3/coins/{}/tickers'.format(ids))
        get_tickers.append(tickers_by_id.json())

    get_tickers = {item['name']:item for item in get_tickers}

    get_markets = []
    for keys, values in get_tickers.items():
        v = values['tickers']
        get_markets.append(v)

    get_markets = pd.DataFrame(get_markets, index = ['Bitcoin', 'Ethereum', 'Litecoin', 'NEO', 'Stellar', 'Ethereum Classic']).T
    cols = get_markets.columns.tolist()

    markets_coins = {}
    for c in cols:
        lists = []
        for row in get_markets[c]:
            if row['market']['name'] in lists: pass
            else: lists.append(row['market']['name'])
        markets_coins[c] = lists
    markets_coins = pd.DataFrame(list(markets_coins.items()), columns = ['Coin', 'Exchanges'])
    markets_coins.insert(loc = 0, column = 'IDs', value = list_selected_coins.keys())
    return markets_coins

markets_coins = get_markets_coins()
markets_coins.head()

,IDs,Coin,Exchanges
0,bitcoin,Bitcoin,"[bitFlyer, FTX (Spot), Kraken, GMO Japan, Coin..."
1,ethereum,Ethereum,"[Binance, Kraken, Huobi US (HBUS), FTX (Spot),..."
2,litecoin,Litecoin,"[Binance, Huobi US (HBUS), Coinbase Pro, Krake..."
3,neo,NEO,"[Bitfinex, Bittrex, Coinone, KuCoin, Upbit, Ga..."
4,stellar,Stellar,"[Bithumb, Coinbase Pro, Upbit, Kraken, Huobi U..."


## Extract and store price data for the past one year from [Deribit](https://www.deribit.com/prinx_chart)

In [12]:
options = webdriver.ChromeOptions()
preferences = {"download.default_directory" : "D:\\Pratik Sharma\\Machine Learning Repository\\API Projects",
               "safebrowsing.enabled": False}
options.add_experimental_option("prefs", preferences)

driver = webdriver.Chrome(options = options, 
                          executable_path = 'D:\\Pratik Sharma\\Machine Learning Repository\\chromedriver.exe')
driver.maximize_window()
driver.get('https://www.deribit.com/')

driver.find_element_by_xpath("""//input[@type = "email"]""").send_keys() #enter
driver.find_element_by_xpath("""//input[@type = "password"]""").send_keys() #enter

driver.find_element_by_name("go").click()

time.sleep(30)

driver.get('https://www.deribit.com/prinx_chart')

element1 = driver.find_element_by_xpath("""//button[@title="1 year"]""")
element2 = driver.find_element_by_xpath("""//div[@class="pull-right context-menu btn-group"]""")
element3 = driver.find_element_by_xpath("""//*[@id="chartmenu"]/li/a""")

hoverover = ActionChains(driver).move_to_element(element1).move_to_element(
    element2).move_to_element(element3).click().perform()

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df = pd.read_csv('2019-10_btc_usd_price_index.csv', low_memory = False, parse_dates = True, header = 1,
             date_parser = dateparse)